<a href="https://colab.research.google.com/github/Paradorn657/CNNs/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install attacut
import tensorflow as tf 
from tensorflow import keras
import random
import json
import numpy as np
from attacut import tokenize
import pickle

import re


with open("/content/sample_data/intents.json",encoding='utf-8') as file:
    data = json.load(file)

In [5]:
import gdown
gdown.download('https://drive.google.com/uc?id=0B0ZXk88koS2KV1FJN0xRX1FxaFE&export=download', 'th.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=0B0ZXk88koS2KV1FJN0xRX1FxaFE&export=download
To: /content/th.zip
122MB [00:02, 53.4MB/s]


'th.zip'

In [6]:
!unzip th.zip

Archive:  th.zip
  inflating: th.bin                  
replace th.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: th.tsv                  


In [7]:
with open('th.tsv',encoding='utf-8') as f:
  vocab = []
  W = None
  vec = ''
  for line in f.readlines():
    temp = line.split('\t')
    if len(temp) == 3:
      vocab.append(temp[1])
      if int(temp[0]) % 1000 == 0:
        print('\r' + temp[0], end='')
      wordvector = np.fromstring(re.sub('\s+', ' ', vec)[1:-1], sep=' ', dtype=np.float32)
      if len(wordvector) > 0:
        if W is None:
          W = wordvector.copy()
        else:
          W = np.vstack((W, wordvector))
      vec = temp[-1]
    elif len(temp) == 1:
      vec += temp[-1]
  W = np.vstack((W, wordvector))

0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  # This is added back by InteractiveShellApp.init_path()


30000

In [8]:
len(vocab)

30225

In [9]:
def sent2idx(x):
  xidx = []
  for w in tokenize(x):
    if w in vocab:
      xidx.append(vocab.index(w))
  return xidx

In [10]:
Xtrain_idx = []
ytrain = []
labels = []
maxlen = 0
for x in data["intents"]:
  print(x["patterns"])
  for word in x["patterns"]:
    Xtrain_idx.append(sent2idx(word))
    if len(Xtrain_idx[-1]) > maxlen:
      maxlen = len(Xtrain_idx[-1])
    ytrain.append(x["tag"])
  if x["tag"] not in labels:
    labels.append(x["tag"])

['ว่าไง', 'สวัสดีครับ', 'มีใครอยู่ไหม', 'หวัดดี', 'ดีครับ']
['บาย', 'เจอกันใหม่', 'ไปละ', 'เจอกัน']
['ขอบคุณค้าบ', 'ขอบใจมาก', 'มันช่วยเหลือเราเยอะเลย']
['เปิดร้านกี่โมง', 'กี่โมงเปิดร้าน', 'เปิดร้านตอนไหน']
['ใช้บัตรเครดิตได้ไหม', 'ใช้มาสเตอร์การ์ดได้ปะ', 'ใช้เงินจ่ายได้แค่ช่องทางเดียวหรอ']


In [ ]:
Xtrain_idx

In [65]:
def setmaxlen(data,maxlen):
  datamaxlen = []
  for i,x in enumerate(data):
    if len(x) < maxlen:
      datamaxlen.append(np.hstack((x,np.zeros(maxlen-len(x)))))
  return datamaxlen

In [66]:
Xtrain_maxlen = setmaxlen(Xtrain_idx,maxlen)

In [67]:
Xtrain_maxlen = np.array(Xtrain_maxlen)
Xtrain_maxlen

array([[1.5000e+01, 6.0790e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [4.7400e+03, 3.4600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [7.0000e+00, 7.8100e+02, 2.4000e+01, 1.4280e+03, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [4.1880e+03, 7.7000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [7.7000e+01, 3.4600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [4.1640e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.0410e+03, 4.0000e+01, 1.3200e+02, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.7000e+01, 6.0800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.0410e+03, 4.0000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [2.2610e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0

In [42]:
ytrain

['ทักทาย',
 'ทักทาย',
 'ทักทาย',
 'ทักทาย',
 'ทักทาย',
 'การจากลา',
 'การจากลา',
 'การจากลา',
 'การจากลา',
 'ขอบคุณ',
 'ขอบคุณ',
 'ขอบคุณ',
 'เวลาเปิดร้าน',
 'เวลาเปิดร้าน',
 'เวลาเปิดร้าน',
 'ช่องทางจ่ายตัง',
 'ช่องทางจ่ายตัง',
 'ช่องทางจ่ายตัง']

In [22]:
ytrain_onehot = []
for i in ytrain:
  bag = [0 for _ in range(len(labels))]
  bag[labels.index(i)] = 1
  ytrain_onehot.append(bag)
ytrain_onehot = np.array(ytrain_onehot)
ytrain_onehot

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [27]:
model.get_layer('embed').set_weights([W])
model.get_layer('embed').trainable = False

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(W.shape[0], W.shape[1], name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(8, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(4)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(5,activation="softmax"))

model.compile(loss=keras.losses.mean_squared_error, optimizer=keras.optimizers.Adam())

model.fit(Xtrain_maxlen,ytrain_onehot[:17], epochs=500)

In [60]:
test_idx = []
test = "จ่ายตัง"
test_idx.append(sent2idx(test))
test_idx

[[1183, 3286]]

In [68]:
test = setmaxlen(test_idx,maxlen)
test

[array([1183., 3286.,    0.,    0.,    0.,    0.,    0.])]

In [47]:
labels

['ทักทาย', 'การจากลา', 'ขอบคุณ', 'เวลาเปิดร้าน', 'ช่องทางจ่ายตัง']

In [63]:
model.predict(test)

array([[0.15185165, 0.18994288, 0.22474614, 0.17496692, 0.2584924 ],
       [0.14698254, 0.18514676, 0.2238651 , 0.18142454, 0.26258102],
       [0.13214552, 0.16310799, 0.38691843, 0.1751777 , 0.14265041],
       [0.13214552, 0.16310799, 0.38691843, 0.1751777 , 0.14265041],
       [0.13214552, 0.16310799, 0.38691843, 0.1751777 , 0.14265041],
       [0.13214552, 0.16310799, 0.38691843, 0.1751777 , 0.14265041],
       [0.13214552, 0.16310799, 0.38691843, 0.1751777 , 0.14265041]],
      dtype=float32)

In [64]:
labels[np.argmax(model.predict(test)[0])]

'ช่องทางจ่ายตัง'